In [ ]:
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install TPOT

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd

# TPOT
from tpot import TPOTClassifier

import catboost
import lightgbm
import xgboost
import sklearn
from sklearn.preprocessing import LabelEncoder
import pickle
from joblib import load, dump

In [ ]:
# # connect with Google Cloud
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/colab/TPS May"
path = r'C:\Users\Chen\Desktop\Kaggle\Classifier\Tabular Playground Series - May 2021'
os.chdir(path)

In [ ]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
# target labelling
f = LabelEncoder()
f.fit(df_train['target'])
y = f.transform(df_train['target'])
y = pd.DataFrame(y).astype('int64')

col = [i for i in df_train.columns if i not in ['id', 'target']]
x = df_train[col]
x = x.astype('int64')

In [ ]:
# construct model
PROJECT_NAME = 'tpot-20210529'
gen = 100
cv = 10
stop = int(gen/5)
model = TPOTRegressor(generations=gen,                    #迭代次數
                      population_size=100,                #初始族群數
                      offspring_size=None,                #子代族群數
                      mutation_rate=0.9,                  #突變機率
                      crossover_rate=0.1,                 #交配機率
                      scoring='neg_log_loss',             #評分標準
                      cv=cv,                              #KFold cross-validation
                      subsample=1,                        #訓練時使用的樣本比例
                      n_jobs=-1,                          #平行運算使用的核心數
                      max_time_mins=None,                 #TPOT最久執行時間
                      max_eval_time_mins=5,               #TPOT中的單一模型最久執行時間
                      random_state=20210529,              #seed
                      config_dict='TPOT cuML',            #TPOT搜尋最佳模型的範圍
                      early_stop=stop,                    #連續多少世代沒進步就提早結束
                      verbosity=2)

In [ ]:
Model = model.fit(x, y_train)

In [ ]:
best_model = Model.fitted_pipeline_
print(best_model)

In [ ]:
# save best model
pickle.dump(best_model, open(f"{PROJECT_NAME}.sav", 'wb'))

In [ ]:
# load best model
best_model = pickle.load(f"{PROJECT_NAME}.sav")

In [ ]:
# predict probability
x_test = df_test.drop(['id'], axis=1)
x_test = x_test.astype('int64')
result = best_model.predict_proba(x_test)
print(result.head())

In [ ]:
# submission
sub = pd.read_csv('sample_submission.csv')
sub[sub.columns[1:]] = result.iloc[:, 1:]
sub.to_csv(f'{PROJECT_NAME}.csv', index=False)